In [87]:
using CSV, DataFrames
using FileIO, Dates
using DelimitedFiles

include("../region_cube_split/cubes.jl")
include("../region_network_create/network.jl")

add_properties (generic function with 1 method)

In [2]:
romania_full = CSV.read("../data/romania.csv", DataFrame);
romania = romania_full[romania_full.Datetime .> DateTime(1976,1,1,0,0,0),:];

vrancea = romania[(romania.Latitude .>= 45.20) .& (romania.Latitude .<= 46.0) .& 
                    (romania.Longitude .>= 26.0) .& (romania.Longitude .<= 27.0) .&
                    (romania.Depth .>= 50.0) .& (romania.Depth .<= 200.0), :];

In [3]:
vrancea, vrancea_cubes = region_cube_split(vrancea,side=5,energyRelease=true);

In [5]:
MG = create_network(vrancea, vrancea_cubes; edgeWeight=true)
# MG = add_properties(MG, vrancea_cubes)

{1727, 7612} undirected Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [6]:
filename="./motifsTriangle_vrancea_5km.csv"

"./motifsTriangle_vrancea_5km.csv"

In [7]:
# motifs = CSV.read(filename, DataFrame;header=false)

# Creates a matrix: number of motifs x motif nodes (3 for triangles, 4 for tetrahedrons)
motifs = readdlm(filename, ',', Int64);
motifs;


In [86]:
typeof(motifs)

Matrix{Int64} (alias for Array{Int64, 2})

In [62]:
# Calculate total energy and mean energy only for cubes that are part of motifs
# This is done to reduce calculations. I do not need to calculate these for each motif
# (because, nodes are repeated in motifs.)
# (I calculate separately, then I just add them up for each motif)

# Get all the cubes, uniquely
vec_motifs = unique!(vec(motifs));

# Will apend to a dictionary: cubeIndex => [totalEnergy, meanEnergy]
df = Dict()
for value in vec_motifs
    # Get the cube from df_cubes
    respective_cube = vrancea_cubes.cubeIndex[value]
    # Use the cube to get all the quakes in that cube from df
    quakesInNode = vrancea[vrancea.cubeIndex .== respective_cube,:];

    # Calculate total energy and mean energy
    totalEnergy = sum(quakesInNode.energyRelease)
    meanEnergy = totalEnergy / length(quakesInNode.energyRelease)

    # Push to dictionary
    df[value] = [totalEnergy, meanEnergy]
end

In [85]:
# Caclulate total energy and mean energy per each motif
motifs_energy = Dict()
for i=1:length(motifs[:,1])
    total_energy_in_motif = df[motifs[i,1]][1] + 
                            df[motifs[i,2]][1] + 
                            df[motifs[i,3]][1]
    mean_energy_in_motif = df[motifs[i,1]][2] + 
                            df[motifs[i,2]][2] + 
                            df[motifs[i,3]][2]

    motifs_energy[i] = [total_energy_in_motif,mean_energy_in_motif]
end

In [84]:
motifs_energy

Dict{Any, Any} with 2312 entries:
  1144 => [1.06164e12, 4.3573e10]
  2108 => [3.06265e12, 1.03199e11]
  1175 => [3.11695e12, 1.12903e11]
  1953 => [2.19417e12, 8.58461e10]
  719  => [4.99602e11, 4.36058e10]
  1546 => [1.3779e12, 6.83807e10]
  1703 => [6.48054e12, 1.74651e11]
  1956 => [2.49914e12, 1.37161e11]
  2261 => [3.07928e12, 9.16835e10]
  2288 => [6.98603e12, 1.94934e11]
  1028 => [9.37478e11, 5.65458e10]
  699  => [2.09455e12, 6.99485e10]
  831  => [1.03681e12, 3.69834e10]
  1299 => [1.29191e12, 8.8249e10]
  1438 => [1.2565e12, 8.09358e10]
  1074 => [1.58424e12, 5.54168e10]
  319  => [4.62493e12, 1.27889e11]
  687  => [1.14613e12, 1.0504e11]
  1812 => [3.50642e12, 1.46098e11]
  ⋮    => ⋮

In [ ]:
using Geodesy

In [88]:
motifs[1,1]

1165

In [ ]:
# Caclulate total energy and mean energy per each motif
areas = Dict()
for i=1:length(motifs[:,1])
    total_energy_in_motif = df[motifs[i,1]][1] + 
                            df[motifs[i,2]][1] + 
                            df[motifs[i,3]][1]
    mean_energy_in_motif = df[motifs[i,1]][2] + df[motifs[i,2]][2] + df[motifs[i,3]][2]

    areas[i] = 
end